In [1]:
import torch
import torch.nn as nn
import snntorch as snn
import torch.nn.functional as nnfunc
from torch.utils.data import DataLoader, TensorDataset
import ExpNeuron as en

In [2]:
class Neuron(nn.Module):
	def __init__(self, beta, threshold, membrane_min):
		super().__init__()
		self.nrn = en.ExpNeuron(
			beta= beta,
			threshold= threshold,
			membrane_zero= membrane_min,
			membrane_min= membrane_min,
			learn_beta= True,
			learn_membrane_min= True,
			learn_threshold= True)
		
	
	def forward(self, spk_input):
		mem = self.nrn.init_neuron()
		spk_outpt = []
		steps = spk_input.shape[1]

		for step in range(steps):
			spk, mem = self.nrn(spk_input[:, step])
			spk_outpt.append(spk)

		spk_outpt = torch.stack(spk_outpt, dim = 1) 

		return spk_outpt
	
	
def gen_spike_train(lambda_, num_steps):
	spike_train = torch.tensor([1 if lambda_ > element else 0 for element in torch.rand(num_steps)], dtype=torch.float32)
	return spike_train

In [3]:
Lambda = 0.95
steps = 100
samples = 100
test_samples = 200
eps = 5e-2
# не забыть вернуть
btch_sz = 5


gnrtr = torch.Generator().manual_seed(0)

# Generating data in (a, b))
a = 0.8
b = 1
if a > 0.8:
	low_lambda_samples = int(samples * 0.3)
	low_lambda_arr = torch.ones(low_lambda_samples).uniform_(0, 0.5)
	lambd_arr = torch.cat((torch.ones(samples - low_lambda_samples).uniform_(a, b), low_lambda_arr))
else:
	lambd_arr = torch.ones(samples).uniform_(a, b)

test_lambd_arr = torch.ones(test_samples).uniform_(0, 1)

test_labels = ((abs(test_lambd_arr - Lambda) <= eps ) | (test_lambd_arr > Lambda)).float().unsqueeze(1)
labels = ((abs(lambd_arr - Lambda) <= eps ) | (lambd_arr > Lambda)).float().unsqueeze(1)

test_trains = torch.stack([gen_spike_train(lambd.item(), steps) for lambd in test_lambd_arr])
trains = torch.stack([gen_spike_train(lambd.item(), steps) for lambd in lambd_arr])

# train_data, test_data = random_split(TensorDataset(trains, labels), [samples * 80 // 100, samples * 20 // 100])
train_data = TensorDataset(trains, labels)
train_dataldr = DataLoader(dataset= train_data, batch_size= btch_sz)

test_data = TensorDataset(test_trains, test_labels) 
test_dataldr = DataLoader(dataset = test_data, batch_size= 1)

In [4]:
beta = torch.tensor(0.5)
threshold = torch.tensor(1.0)
membrane_min = torch.tensor(0.5)

lrng_rt = 1e-3

epochs = 50

neuron = Neuron(beta = beta, threshold = threshold, membrane_min = membrane_min)
optim = torch.optim.Adam(neuron.parameters(), lr = lrng_rt)

In [5]:
for epoch in range(epochs):
		for trns, lbls in train_dataldr:
				optim.zero_grad()

				outputs = neuron(trns)
				spike_cnt = outputs.mean(dim=1)
				# При Lambda > 0.9 и Lambda < 0.1 достоверность результатов падает

				loss = nnfunc.binary_cross_entropy(input= spike_cnt, target= lbls.squeeze(), reduction= "mean")

				loss.backward()

				optim.step()
	
				# В данной модели beta принимает значения [0, 1]
				if beta.item() > 1:
					beta = torch.tensor(0.999)
				if beta.item() < 0:
					beta = torch.tensor(0.01)

		if epoch % 5 == 0:
			print((f"Epoch {epoch}, Loss: {loss.item():.4f}, "
				f"Beta: {neuron.nrn.beta.item():.4f}, Threshold: {neuron.nrn.threshold.item():.4f}"))
		


# Testing accuracy
data = []
beta = torch.tensor(0.3234)
threshold = torch.tensor(0.6971)
for trn, lbl in test_dataldr:
		prediction = neuron(trn).mean(dim=1).item()
		data.append([float(prediction > 0.5), lbl.item()])
		# print((f"pred: {prediction:.2f}, predicted: {prediction > 0.5}, "
		#        f"true: {lbl}"))
data = torch.tensor(data)
true_positive = data[(data[:, 0] == 1) & (data[:, 1] == 1), 0].numel()
false_positive = data[(data[:, 0] == 1) & (data[:, 1] == 0), 0].numel()
true_negative = data[(data[:, 0] == 0) & (data[:, 1] == 0), 0].numel()
false_negative = data[(data[:, 0] == 0) & (data[:, 1] == 1), 0].numel()

if true_positive and true_negative: 
		precision = true_positive / (true_positive + false_positive)
		recall = true_positive / (true_positive + false_negative)
		accuracy = (true_positive + true_negative) / (true_positive + true_negative 
																								+ false_positive + false_negative)

		print((f"precision: {precision:.2f}, recall: {recall:.2f}, accuracy : {accuracy:.2f}"))
		print((f"\ntrue_positive: {true_positive},\n"
				f"true_negative: {true_negative},\nfalse_negative: {false_negative},\n"
				f"false_positive: {false_positive}"))
else:
		print((f"Low accuracy of the model.\ntrue_positive: {true_positive},\n"
				f"true_negative: {true_negative},\nfalse_negative: {false_negative},\n"
				f"false_positive: {false_positive}"))
	
if precision and recall:
		f1 = 2 * precision * recall / (precision + recall)
		print(f"f1 metric: {f1:.3f}")

tensor([1., 1., 1., 1., 1.]) 0 tensor(0.5000)
correcting shape
tensor([1., 0., 1., 0., 1.]) tensor([1., 1., 1., 1., 1.]) tensor([0.5000, 0.5000, 0.5000, 0.5000, 0.5000], grad_fn=<CopyBackwards>)
tensor([1., 1., 1., 1., 1.]) tensor([1., 1., 1., 1., 1.]) tensor([0.5000, 0.5000, 0.5000, 0.5000, 0.5000], grad_fn=<CopyBackwards>)
tensor([1., 1., 1., 1., 0.]) tensor([1., 2., 1., 2., 1.]) tensor([0.5000, 0.5000, 0.5000, 0.5000, 0.5000], grad_fn=<CopyBackwards>)
tensor([1., 0., 1., 1., 1.]) tensor([1., 3., 1., 3., 1.]) tensor([0.5000, 0.5000, 0.5000, 0.5000, 0.5000], grad_fn=<CopyBackwards>)
tensor([1., 1., 0., 1., 0.]) tensor([1., 1., 1., 1., 2.]) tensor([0.5000, 0.5000, 0.5000, 0.5000, 0.5000], grad_fn=<CopyBackwards>)
tensor([1., 0., 1., 1., 1.]) tensor([1., 2., 1., 1., 3.]) tensor([0.5000, 0.5000, 0.5000, 0.5000, 0.5000], grad_fn=<CopyBackwards>)
tensor([1., 1., 1., 1., 0.]) tensor([1., 3., 2., 1., 4.]) tensor([0.5000, 0.5000, 0.5000, 0.5000, 0.5000], grad_fn=<CopyBackwards>)
tensor([1., 1

RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor [5]] is at version 200; expected version 199 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).

In [ ]:
for par, val in neuron.named_parameters():
	print(par, val)

nrn.threshold Parameter containing:
tensor(1., requires_grad=True)
nrn.beta Parameter containing:
tensor(0.5000, requires_grad=True)
nrn.membrane_min Parameter containing:
tensor(0.5000, requires_grad=True)
